# The AbspectroscoPY workflow: files required

This file requires the following files:

I.   TOOLBOX ("abspectroscopy_functions.py")

II.  USER CONFIGURATION FILE ("config.py") 
     The file provides an example of configuration settings for the variables used in the functions that the user can modify.
     
III. CSV FILES from the folders "data_scan_fp" and "other_data".

# The AbspectroscoPY workflow: contents

I. IMPORT RAW DATA FILES

II. PREPROCESS THE DATASET

    A) DATA TYPE CONVERSION
    B) DATA QUALITY ASSESSMENT
    C) TIME AXIS SHIFTING
    D) ATTENUATION DATA CORRECTION
    E) DATA SMOOTHING
    
III. EXPLORE THE DATASET
    
    A) VISUALISATION OF DATA DISTRIBUTION
    B) OUTLIER/EVENT IDENTIFICATION AND REMOVAL
    
IV. INTERPRET THE RESULTS

    A) ABSORBANCE RATIOS
    B) ABSORBANCE SPECTRA CHANGES

# Import packages, user configuration and AbspectroscoPY toolbox

In [ ]:
import tkinter as tk
from tkinter import messagebox
import sys
import os   
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from matplotlib.ticker import (MultipleLocator, FormatStrFormatter, AutoMinorLocator)
from datetime import datetime
from datetime import timedelta
from datetime import datetime as dt
import seaborn as sns
from scipy import interpolate
from sklearn.linear_model import LinearRegression
import scipy as sp 
import statistics 
from statistics import median
from scipy.optimize import curve_fit
from pylab import *
import glob

from config import *                     # Personal settings of local user
import abspectroscopy_functions as abspy # Functions from the AbspectroscoPY toolbox

# Specify the file location, generate the result folders and specify the following variables

In [ ]:
### FILE LOCATION
pathwithlibraries = 'C:/Users/cace0002/AbspectroscoPY/'
indata = os.path.join(pathwithlibraries,'data_scan_fp/')        # path where to search for the data
indata_events = os.path.join(pathwithlibraries,'other_data/events_table.csv') # input event table

### GENERAL VARIABLES
col_sel = '255 nm'                                              # select a specific wavelength to plot
timestart ='2018-11-12 16:04:00'                                # starting date
timeend ='2018-12-03 20:44:00'                                  # ending date

### TIME-AXIS SHIFTING
nsamples_per_hour = 30                                          # sampling frequency (number of samples per hour)
tshift = '0 hours 28 min'                                       # time shift not due to when the Daylight Saving Time ends
tshift2 = '12 hours 00 min'                                     # time shift to compare measurements from different sensors

### ATTENUATION DATA CORRECTION
path_length = 3.5                                               # path length of the window of the sensor [cm]

### OUTLIER/EVENT IDENTIFICATION AND REMOVAL
splitstrs = ['2018-11-19 00:00:00', '2018-11-24 12:00:00']      # specify the dates you want to use to split the dataset in periods for the IQR method

### INTERPRET THE RESULTS
# abs_ratio:
date_ref_start = '2018-11-13 05:02:00'                          # define reference period for computing the changes (%) ratio of absorbance data and spectral curve data
date_ref_end = '2018-11-14 05:02:00'
date_interesting = '2018-12-04 08:38:00'                        # define a date to look at the change in percentage at a certain date compared to the reference period

# abs_spectral_curve
col_sel_sc = '254.5'                                            # column to plot
start_date = '2018-11-13 05:02:00'                              # define the range of dates for which we want to generate the spectral curve
end_date = '2018-12-04 08:44:00'
year = 'november_december_2018'                                 # title when saving the csv files

## Check

In [ ]:
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

MsgBox = tk.messagebox.askquestion ('Check','Have you specified the variables and the file location in cell 2?',icon = 'warning')
if MsgBox == 'yes':  
    root.destroy()
else:
    tk.messagebox.showwarning('Warning','Please specify them and restart the kernel!')  
    root.destroy()
    raise SystemExit('Stop right here and restart the kernel')

In [ ]:
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

MsgBox = tk.messagebox.askquestion ('Check','Do you know that, besides these variables, you might have to modify the configuration file to adapt the scripts to your data (config.py)?',icon = 'question')
if MsgBox == 'yes':  
    root.destroy()
else:
    tk.messagebox.showinfo ( "Info", "Please check and edit the file config.py")   
    root.destroy()
    
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

MsgBox = tk.messagebox.askquestion ('Check','Do you know that there is a file where to modify the functions, if it is not enough changing the configuration file (abspectroscopy_functions.py)?',icon = 'question')
root.destroy()

### From here onwards no user input should be required, except for tshift_dst and abs_path

In [ ]:
output = os.path.join(pathwithlibraries,'results/')                         # create a new folder where to store the results
if not os.path.exists(output):
    os.mkdir(output)
    
output_outliers = os.path.join(output,'sr_periods_outliers/')               # create a new folder that will include the outlier files
output_no_outliers = os.path.join(output,'sr_periods_no_outliers/')         # create a new folder that will include the files without outliers
if not os.path.exists(output_outliers): 
    os.mkdir(output_outliers)
if not os.path.exists(output_no_outliers):
    os.mkdir(output_no_outliers)

# I. IMPORT RAW DATA FILES

## DATASET ASSEMBLY

## abs_read

In [ ]:
listoffileswithpath,listoffilesnopath = abspy.get_files_list(indata) # get the list of files with a specific pattern
print('List of files including path:', '\n', listoffileswithpath, '\n')
print('List of files without path:', '\n', listoffilesnopath, '\n')

dateheadername = abspy.guess_date_column(listoffileswithpath, possibledateheadernames, header_rownr+1) # determine the name of the date column using a list of possible date column names 
print('Date header name:', dateheadername)

In [ ]:
df_init = abspy.abs_read(listoffileswithpath, listoffilesnopath, header_rownr, dateheadername, drop_col) # import the list of files
df_init.to_csv(output + 'df_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index=True) # export the dataset

df_init.head()

# II. PREPROCESS THE DATASET

## A) DATA TYPE CONVERSION

## convert2dtype

In [ ]:
df_dtypecor = abspy.convert2dtype(df_init, dateheadername)
df_dtypecor.to_csv(output + 'df_dtypecor_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index = True) # to export the type-converted dataset
df_dtypecor

## B) DATA QUALITY ASSESSMENT

## **dropna

In [ ]:
df_nonan = df_dtypecor.copy()
df_nonan = df_nonan.drop(df_nonan[df_nonan.isnull().all(axis=1)].index)  # drop rows containing only missing data
df_nonan = df_nonan.dropna(axis=1, how='all', inplace=False) # drop columns containing only missing data
df_nonan

## nan_check

In [ ]:
nan_colper, nan_rowper = abspy.nan_check(df_nonan, dateheadername)
nan_colper.to_csv(output + 'missing_data_per_column_percent.csv', sep = sep, decimal = decimal, index=True) # export the missing data per column and row as percentage
nan_rowper.to_csv(output + 'missing_data_per_row_percent.csv', sep = sep, decimal = decimal, index=True)
df_nonan.to_csv(output + 'df_nonan_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index=True) # export the dataframe without missing data
nan_colper, nan_rowper

## dup_check

In [ ]:
df_dup, df_dup_all = abspy.dup_check(df_nonan, dateheadername)
df_dup.to_csv(output + 'duplicates_by_date.csv', sep = sep, decimal = decimal, index=True) # export the duplicates by dateheadername 
df_dup_all.to_csv(output + 'duplicates.csv', sep = sep, decimal = decimal, index=True) # export the duplicates by all columns    
df_dup, df_dup_all

In [ ]:
%matplotlib notebook 
plt.ion()

# Run twice if the plot looks too small

title1 =  'duplicates_'
abspy.makeaplot(df_dup_all, output, col_sel, timestart, timeend, sample_name, title1) 

## **drop_duplicates

In [ ]:
df_nodupall = df_nonan.drop_duplicates(subset=None, keep="first", inplace=False) # drop second duplicate by all columns
df_nodupall.to_csv(output + 'df_nodup_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index=True) # export the dataframe
df_nodupall

## C) TIME AXIS SHIFTING

## tshift_dst

In [ ]:
df_dst = df_nodupall.copy()

In [ ]:
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

MsgBox = tk.messagebox.showwarning ('Warning','Do the data need to be time-shifted by Daylight Saving Time? Input yes or no in the following cell.', icon = 'warning')
root.destroy()

In [ ]:
answer = input()
if answer == 'yes':
    df_dst = abspy.tshift_dst(df_nodupall, dateheadername, nsamples_per_hour)
    df_dst.to_csv(output + 'df_dst.csv', sep = sep, decimal = decimal, index=True) # export the dst time shifted dataset  
    df_shifted = df_dst.copy()    
elif answer == 'no':
    df_shifted = df_dst.copy()
else:
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", True)

    MsgBox = tk.messagebox.showerror ('Error','You are not providing one of the two possible answers. Please input "yes" or "no".',icon = 'error')
    root.destroy()    

## **timedelta

#### Skip the next two cells if there is no extra time difference:

#### a) shift the dataset in time, if there is any time difference between the sensor and the clock not due to when the Daylight Saving Time ends

In [ ]:
timeshift = pd.Timedelta(tshift)
df_shifted.index = df_shifted.index + timeshift
df_shifted

#### b) to be able to compare the sensors data of the surface water to the ones inside the plant shift the time one hour forward and account for the time the surface water needs to reach the treatment step (e.g. 11 hours)

In [ ]:
timeshift2 = pd.Timedelta(tshift2)
df_shifted.index = df_shifted.index + timeshift2
df_shifted.to_csv(output + 'df_shifted_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index=True) # export the time shifted dataset 
df_shifted

## D) ATTENUATION DATA CORRECTION

## abs_pathcor

In [ ]:
abspy.abs_pathcor(df_shifted, path_length) [0:4]

In [ ]:
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

MsgBox = tk.messagebox.showwarning ('Warning','Compare the attenuation value read by the sensor before and after path length correction for a specific date to the value obtained in the laboratory. Input yes or no in the following cell.',icon = 'warning')
root.destroy()

In [ ]:
answer = input()
if answer == 'yes':
    df_pathcor = abspy.abs_pathcor(df_shifted, path_length)[5] # the dataframe with corrected values is the fifth output of the function
elif answer == 'no':
    df_pathcor = df_shifted.copy()
else:
    root = tk.Tk()
    root.withdraw()
    root.attributes("-topmost", True)

    MsgBox = tk.messagebox.showerror ('Error','You are not providing one of the two possible answers. Please input "yes" or "no".',icon = 'error')
    root.destroy()    

In [ ]:
df_pathcor.to_csv(output + 'df_pathcor_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index=True) # export the pathlength corrected dataset 
df_pathcor

## abs_basecor

#### Plot the absorbance spectra for different dates covering the temporal variability of the data and choose a wavelength range to correct for the baseline drift

In [ ]:
%matplotlib notebook
plt.ion()
abspy.makeabsplot(df_pathcor, output, dateparsingformat, nperiods, sample_name)

#### Perform the baseline correction

In [ ]:
df_bc = abspy.abs_basecor(df_pathcor, startwv)
df_bc.to_csv(output + 'df_baselinecor_' + str(sample_name) + '.csv', sep = sep, decimal = decimal, index=True) # export the dataframe
df_bc

## E) DATA SMOOTHING

## **rolling

In [ ]:
sample_frequency_min = 60/nsamples_per_hour                # sampling frequency (measurement interval in minutes)

median_window1 = median_window1_min / sample_frequency_min # median window from minute to number of samples
median_window2 = median_window2_min / sample_frequency_min
median_window3 = median_window3_min / sample_frequency_min
median_window_selected = median_window_min_selected / sample_frequency_min
print('number of samples for window 1:', median_window1)
print('number of samples for window 2:', median_window2)
print('number of samples for window 3:', median_window3)
print('number of samples for window selected:', median_window_selected)

df_median1 = df_bc.copy()
median_window1_int=int(median_window1)
df_median1 = df_median1.rolling(median_window1_int, center=True, axis=0).median()# compute the rolling median of the absorbance series
df_median1 = df_median1.dropna(axis=0)                                           # drop rows with NaN due to rolling median calculation

df_median2 = df_bc.copy()
median_window2_int=int(median_window2)
df_median2 = df_median2.rolling(median_window2_int, center=True, axis=0).median()
df_median2 = df_median2.dropna(axis=0) 

df_median3 = df_bc.copy()
median_window3_int=int(median_window3)
df_median3 = df_median3.rolling(median_window3_int, center=True, axis=0).median()
df_median3 = df_median3.dropna(axis=0)

In [ ]:
%matplotlib notebook
plt.ion()
abspy.makerollplot(df_median1, df_median2, df_median3, output, col_sel, timestart, timeend, sample_name)

In [ ]:
df_median_selected = df_bc.copy()
median_window_selected_int=int(median_window_selected)
df_median_selected = df_median_selected.rolling(median_window_selected_int, center=True, axis=0).median()
df_median_selected = df_median_selected.dropna(axis=0) 
df_median_selected.to_csv(output + 'df_mediancor_' + str(sample_name) + '_' + str(median_window_selected) + '.csv', sep = sep, decimal = decimal, index=True) # export the dataframe

# III. EXPLORE THE DATASET

## A) VISUALISATION OF DATA DISTRIBUTION

## **kdeplot

In [ ]:
df_median = df_median_selected.copy()
%matplotlib notebook
plt.ion()
abspy.makeakdeplot(df_median_selected, output, sample_name)

## B) OUTLIER/EVENT IDENTIFICATION AND REMOVAL

## abs_slope_ratio

In [ ]:
df_sr = abspy.abs_slope_ratio(df_median, S_f)
df_sr[1:].to_csv(output + 'df_sr_' + str(sample_name) + '_' + str(S_f) +'.csv', index = True, sep = ';') # export the  dataframe

In [ ]:
%matplotlib notebook
plt.ion()
title1 = 'slope_ratio_'
abspy.makeaplot(df_sr, output, 'SR', timestart, timeend, sample_name, title1, ylabel = 'Slope ratio [dimensionless]')

## outlier_id_drop_iqr

In [ ]:
df_sr2 = df_sr.copy()
dflist, out1, out2 = abspy.outlier_id_drop_iqr(df_sr2, output_no_outliers, output_outliers, splitstrs, timestart, timeend, dateparsingformat, sample_name)
print('Lower and upper limits of the interquartile range:', '\n', out1, '\n', 'outlier percentage:', out2)

In [ ]:
abspy.makeaoutplot(dflist, output_no_outliers, output, sr_col, timestart, timeend, dateparsingformat, splitstrs, sample_name)

## outlier_id_drop

In [ ]:
df_ev, df_drop = abspy.outliers_id_drop(df_bc, indata_events, output, col_sel)

In [ ]:
%matplotlib notebook
plt.ion()
title1 = 'absorbance_data_baseline_corrected_with_events'  
abspy.makeaplotev(df_bc, df_ev, output, col_sel, timestart, timeend, dateparsingformat, sample_name, title1)

In [ ]:
%matplotlib notebook
plt.ion()
title2 = 'absorbance_data_baseline_corrected_with_no_events' 
abspy.makeaplotev(df_drop, df_ev, output, col_sel, timestart, timeend, dateparsingformat, sample_name, title2)

# IV. INTERPRET THE RESULTS

## abs_ratio

In [ ]:
df_ratio, df_change_per = abspy.abs_ratio(df_median, date_ref_start, date_ref_end, dateparsingformat, wv1, wv2, date_interesting)
df_change_per.to_csv(output + 'df_absorbance_ratio_change_' + str(wv1) + '_' + str(wv2) + '.csv', index = True) # export the dataframe

In [ ]:
%matplotlib notebook
plt.ion()
title1 = 'absorbance_ratio' + str(wv1) + '_' + str(wv2)
abspy.makeaplot_nocolsel(df_change_per, output, timestart, timeend, wv1, wv2, sample_name, title1) 

In [ ]:
%matplotlib notebook
plt.ion()
title2 = 'absorbance_ratio_change_' + str(wv1) + '_' + str(wv2)
abspy.makeaplot_nocolsel(df_change_per, output, timestart, timeend, wv1, wv2, sample_name, title2, ylabel = '{} {} '.format('A' + '$_{' + wv1 + '}$' + '/' + 'A' + '$_{' + wv2 + '}$', 'change [%]')) 

## abs_fit_exponential

In [ ]:
exp_par_no_zero, exp_par_std_no_zero = abspy.abs_fit_exponential(df_median, startwl, endwl, wl0, S_f)
exp_par_no_zero[1:].to_csv(output +'exp_fit_' + str(sample_name) + '_' + str(wl0) + '_' + str(S_f) +'.csv', index = True, sep = ';')
exp_par_std_no_zero[1:].to_csv(output + 'exp_fit_std_' + str(sample_name) + '_' + str(wl0) + '_' + str(S_f) +'.csv', index = True, sep =';')     

In [ ]:
%matplotlib notebook
plt.ion()
abspy.abs_fit_exponential_plot(df_median, output, dateparsingformat, date_interesting, startwl, endwl, wl0)

## abs_spectral_curve

In [ ]:
sc_data_r2, sc_data_r2_no_NaN = abspy.abs_spectral_curve(df_median, start_date, end_date, dateparsingformat, S_f, r2threshold)
sc_data_r2.to_csv(output + 'df_sc_' + str(sample_name) + '_' + str(S_f) + '_' + str(year) +'.csv', index = True, sep = ';')
sc_data_r2_no_NaN.to_csv(output + 'df_sc_no_NaN_' + str(sample_name) + '_' + str(S_f) + '_' + str(year) +'.csv', index = True, sep = ';') # to drop the columns with any missing values

In [ ]:
%matplotlib notebook
plt.ion()
title1 = 'spectral_curve_'
abspy.makeaplot(sc_data_r2_no_NaN, output, col_sel_sc, timestart, timeend, sample_name, title1, ylabel = 'Negative spectral slope [nm $^{-1}$]')

#### Spectral slope changes [%]

In [ ]:
df_sc_change_per = abspy.abs_spectral_curve_perchanges(sc_data_r2_no_NaN, date_ref_start, date_ref_end, dateparsingformat)
df_sc_change_per.to_csv(output + 'df_sc_change_' + str(sample_name) + '_' + str(S_f) + '_' + str(year) +'.csv', index = True) # export the dataframe

In [ ]:
%matplotlib notebook
plt.ion()
title2 = 'spectral_curve_change_'
abspy.makeaplot(df_sc_change_per, output, col_sel_sc, timestart, timeend, sample_name, title2, ylabel = 'Negative spectral slope change [%]')

In [ ]:
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

MsgBox = tk.messagebox.showinfo ('Info','If the plots are not visible or if they are too small run again the individual cell.',icon = 'info')
root.destroy()

In [ ]:
root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

MsgBox = tk.messagebox.showinfo ('Info', 'Thank you for using AbspectroscoPY. If you find any bug or you would like to improve the scripts, please propose changes on GitHub or write to claudia.cascone@gmail.com.', icon = 'info')
root.destroy()